# Attributing population to street segments within OAs

We can get OA-level population data from 2021 census, and use it to estimate populations at other spatial scales, 
naively assuming an even population distribution. But this will be wildly inaccurate for large rural OAs where the 
population is concentrated in one or more parts of the OA, and other parts of the OA are lakes, rivers or countryside.

To address this we make the assumption that all the population is located on the (drivable) street network. Steps are 
as follows: 

- For each OA, get the street network within the OA
- Get the population count and divide this evenly onto each street segment according to its length (preserving whole 
values)
- Sample points randomly within each street segment
- Generate the features the population needs to be attributed to (e.g. a regular square grid)
- Count the number of population points that intersect each of the new features

One issue is that we can either get street segments that are wholly within the OA (which omits street segments, and 
over-concentrates the population) or get all that are partly within the OA (which means we attribute population 
outside the OA). On balance the former is probably better for our purposes

Toby might have some clever ideas... yes he does - `gpd.overlay` 

In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
import geopandas as gpd
import humanleague as hl

from spatial import get_force_boundary, get_hex_grid, get_street_network, map_to_spatial_unit
from utils import Month, load_crime_data, monthgen

In [7]:
FORCE = "West Yorkshire"
CRIME_TYPE = "Anti-social behaviour"
END_MONTH = Month(2025, 5)

In [8]:
boundary = get_force_boundary(FORCE)
raw_crime_data = load_crime_data(FORCE, monthgen(END_MONTH - 36, end=END_MONTH), filters={"Crime type": CRIME_TYPE})
crime_data, features = map_to_spatial_unit(raw_crime_data, boundary, "OA", resolution="FE")

In [10]:
count_data = features.join(crime_data.groupby("spatial_unit").size().rename("count"), how="right")
count_data

,LSOA21CD,LSOA21NM,LSOA21NMW,BNG_E,BNG_N,LAT,LONG,geometry,count
spatial_unit,,,,,,,,,
E00053353,E01010573,Bradford 013A,None,413586,440190,53.8578,-1.79493,"POLYGON ((415817.093 440872.597, 415821.094 44...",22
E00053354,E01010573,Bradford 013A,None,414822,439839,53.8546,-1.77615,"POLYGON ((415078 439967.001, 415058.323 439954...",2
E00053356,E01010576,Bradford 016E,None,416512,439407,53.8507,-1.75049,"POLYGON ((416668 439392.028, 416667.653 439391...",13
E00053357,E01010573,Bradford 013A,None,414791,439080,53.8478,-1.77666,"POLYGON ((415143.909 439176.235, 415143 439175...",19
E00053358,E01010568,Bradford 016A,None,416417,438807,53.8453,-1.75196,"POLYGON ((416497.602 439065.761, 416499 439066...",1
...,...,...,...,...,...,...,...,...,...
E00187149,E01011468,Leeds 112B,None,431667,431748,53.7811,-1.52091,"POLYGON ((432104.145 431535.463, 432083.179 43...",8
E00187150,E01011393,Leeds 103C,None,439268,427386,53.7414,-1.40612,"POLYGON ((439898.904 427922.438, 439803 427876...",11
E00187151,E01011266,Leeds 008A,None,417519,443044,53.8833,-1.73497,"POLYGON ((417609.933 443039.985, 417607.859 44...",4


In [41]:
oa = "E00053954"  # "E00053982"
m = features.loc[[oa]].explore(
    tiles="CartoDB positron",
)
streets = get_street_network(features.loc[[oa]], simplify=True, truncate_by_edge=True)
streets = gpd.overlay(
    streets,
    features.loc[[oa]],
    # how="intersection",
)
streets.explore(color="red", m=m)

In [42]:
m = features.loc[[oa]].explore(
    tiles="CartoDB positron",
)
new_features = get_hex_grid(features.loc[[oa]], size=124.0)
new_features.explore(tiles="CartoDB positron", color="green", m=m)
# split evenly by length, rounding
total_pop = 400
pops, stats = hl.integerise(total_pop * streets.length / streets.length.sum())  # proportion of total length
assert stats["conv"]

pop = streets.sample_points(pops).explode()  # need to explode to count intersections with individual points
pop.explore(tiles="CartoDB positron", color="red", m=m)

In [43]:
m = features.loc[[oa]].explore(
    tiles="CartoDB positron",
)
new_features["pop"] = new_features.geometry.map(lambda g: pop.intersects(g).sum())
new_features.explore("pop", tiles="CartoDB positron", cmap="plasma", m=m)

In [14]:
new_features["pop"].sum()  # == total_pop, "Population distribution does not match total population"

np.int64(346)